In [5]:
import pandas as pd
import re
import numpy as np

In [6]:
post_path = 'post_data.csv'
columns = ['index','id', 'categories', 'description', 'birthday', 'gender', 
           'post_followers', 'post_time', 'post_likes', 'post_interactions', 'post_comments', 
           'post_caption', 'post_hashtags', 'post_mentions']

In [7]:
df = pd.read_csv(post_path)

In [8]:
df.head()

,id,categories,description,birthday,gender,post_followers,post_time,post_likes,post_comments,post_caption,post_hashtags,post_mentions
0,"{:id=>71177, :categories=>[10, 13, 24], :descr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{:id=>71177, :categories=>[10, 13, 24], :descr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{:id=>71177, :categories=>[10, 13, 24], :descr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{:id=>71177, :categories=>[10, 13, 24], :descr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{:id=>71177, :categories=>[10, 13, 24], :descr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
split_categories = [':id=>', ':categories=>', ':description=>', ':birthday=>', ':gender=>', ':post_followers=>', 
                 ':post_time=>', ':post_interactions=>', ':post_likes=>',':post_comments=>',':post_caption=>',
                 ':post_caption=>',':post_hashtags=>',':post_mentions=>']

In [10]:
df2 = pd.DataFrame(df['id'].str.replace('{', ''))

In [11]:
df2 = pd.DataFrame(df2['id'].str.replace('}', ''))

In [12]:
for i in split_categories:
    df2 = pd.DataFrame(df2['id'].str.replace(i, 'œ∑'))

In [13]:
df3 = df2['id'].str.split('œ∑', expand=True)

In [14]:
df3.columns = columns

In [15]:
df=df3

### Id Column: remove comma, convert to int

In [16]:
df['id'] = pd.DataFrame(df['id'].str.replace(',', ''))

In [17]:
df.id = pd.to_numeric(df.id)

### Convert Categories String to list, one hot encode

In [18]:
df['categories']= df['categories'].str.strip('], ').str.strip('[]').str.strip('] ').str.replace(' ', '').str.split(',')

In [19]:
df.categories.head()

0    [10, 13, 24]
1    [10, 13, 24]
2    [10, 13, 24]
3    [10, 13, 24]
4    [10, 13, 24]
Name: categories, dtype: object

In [20]:
s = df['categories']

In [21]:
df_cat = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

Apply category labels to One Hot DF

1 => 'Beauty',
2 => 'DIY',
3 => 'Fashion',
4 => 'Lifestyle',
5 => 'Menswear',
6 => 'Models',
8 => 'News',
9 => 'Personal Style',
10 => 'Photography',
11 => 'Illustration',
12 => 'Vlog',
13 => 'Food',
14 => 'Artist',
15 => 'Musician/DJ',
16 => "Tech",
17 => "Parenting",
18 => "Entertainment",
19 => "Fitness",
20 => "Health and Wellness",
21 => "Automotive",
22 => "Home Decor",
23 => "Art and Design",
24 => "Travel"

In [22]:
cat_columns = ['no_cat','cat_beauty', 'cat_photography', 'cat_illustration', 'cat_vlog', 'cat_food', 'cat_artist', 'cat_musician/DJ', 
               "cat_tech", "cat_parenting", "cat_entertainment", "cat_fitness", 'cat_diy', "cat_health_wellness", "cat_automotive", 
               "cat_home_decor", "cat_art_design", "cat_travel",'cat_fashion', 'cat_lifestyle', 'cat_menswear', 
               'cat_models', 'unknown', 'cat_news', 'cat_personal_style']

In [23]:
df_cat.columns = cat_columns

In [24]:
df = df.join(df_cat)

In [25]:
df.drop('categories', axis=1, inplace=True)

### description: remove quotes comvert to string

In [26]:
df['description'] = pd.DataFrame(df['description'].str.replace('",', ''))

In [27]:
df['description'] = pd.DataFrame(df['description'].str.replace('"', ''))

In [28]:
df['description'] = pd.DataFrame(df['description'].str.replace('"",', ''))

In [29]:
df['description'] = pd.DataFrame(df['description'].replace('nil, ', None))

In [30]:
df.description = df.description.astype(str)

### Convert Birthday to datetime 

In [31]:
df['birthday'] = pd.DataFrame(df['birthday'].str.replace('19992', '1992'))

In [32]:
day_names = ['Mon, ', 'Tue, ', 'Wed, ', 'Thu, ', 'Fri, ', 'Sat, ', 'Sun, ', ', ', 'nil']
for i in day_names:
    df['birthday'] = pd.DataFrame(df['birthday'].str.replace(i, ''))

In [33]:
df['birthday'] = pd.to_datetime(df['birthday'], dayfirst=True)

In [34]:
df['birthday'] = df['birthday'].dt.year

In [35]:
df.birthday.fillna(value=df.birthday.mean(), inplace=True)

### Gender: Remove comma, remove 'nil',  convert to int

In [36]:
df['gender'] = pd.DataFrame(df['gender'].str.replace(',', ''))

In [37]:
df['gender'] = pd.DataFrame(df['gender'].replace('nil ', None))

In [38]:
df.gender = pd.to_numeric(df.gender)

### post_followers: Remove comma,  convert to int

In [39]:
df['post_followers'] = pd.DataFrame(df['post_followers'].str.replace(',', ''))

In [40]:
df.post_followers = pd.to_numeric(df.post_followers)

### post_time: remove quotes, Convert to datetime, 

In [41]:
df['post_time'] = pd.DataFrame(df['post_time'].str.replace('", ', ''))

In [42]:
df['post_time'] = pd.DataFrame(df['post_time'].str.replace('"', ''))

In [43]:
df['post_time'] = pd.to_datetime(df['post_time'], dayfirst=True) #format= '%d %m %Y'

### Reduce to 100 rows per user

In [44]:
df = df.groupby('id').head(100).reset_index(drop=True)

### Post Time bin into DOW & TOD

In [45]:
df['post_day'] = df['post_time'].dt.dayofweek

In [46]:
df['post_hour'] = df['post_time'].dt.hour

In [47]:
df.drop('post_time', axis=1, inplace=True)

### post_likes: Remove commas, convert to int

In [48]:
df['post_likes'] = pd.DataFrame(df['post_likes'].str.replace(',', ''))

In [49]:
df.post_likes = pd.to_numeric(df.post_likes)

### post_interactions: Remove commas convert to int

In [50]:
df['post_interactions'] = pd.DataFrame(df['post_interactions'].str.replace(', ', ''))

In [51]:
df['post_interactions'] = pd.DataFrame(df['post_interactions'].str.replace(',', ''))

In [52]:
df['post_interactions'] = pd.DataFrame(df['post_interactions'].str.replace('nil', ''))

In [53]:
df['post_interactions'] = pd.DataFrame(df['post_interactions'].str.replace('""', ''))

In [54]:
df.post_interactions = pd.to_numeric(df.post_interactions)

### post_comments: Remove commas, convert to int

In [55]:
df['post_comments'] = pd.DataFrame(df['post_comments'].str.replace(', ', ''))

In [56]:
df['post_comments'] = pd.DataFrame(df['post_comments'].str.replace(',', ''))

In [57]:
df['post_comments'] = pd.DataFrame(df['post_comments'].str.replace('nil', ''))

In [58]:
df['post_comments'] = pd.DataFrame(df['post_comments'].str.replace('""', ''))

In [59]:
df.post_comments = pd.to_numeric(df.post_comments)

### post_caption: remove quotes and commas

In [60]:
df['post_caption'] = pd.DataFrame(df['post_caption'].str.replace('",', ''))

In [61]:
df['post_caption'] = pd.DataFrame(df['post_caption'].str.replace('"', ''))

In [62]:
df['post_caption'] = pd.DataFrame(df['post_caption'].str.replace('"",', ''))

In [63]:
df['post_caption'] = pd.DataFrame(df['post_caption'].str.replace('\n', ''))

In [64]:
df.post_caption = df.post_caption.astype(str)

### post_hashtags: Remove quotes, convert to string

In [65]:
df['post_hashtags'] = pd.DataFrame(df['post_hashtags'].str.replace('",', ''))

In [66]:
df['post_hashtags'] = pd.DataFrame(df['post_hashtags'].str.replace('"', ''))

In [67]:
df['post_hashtags'] = pd.DataFrame(df['post_hashtags'].str.replace('"",', ''))

In [68]:
df.post_hashtags = df.post_hashtags.astype(str)

### post_mentions: Remove quotes, convert to string

In [69]:
df['post_mentions'] = pd.DataFrame(df['post_mentions'].str.replace('",', ''))

In [70]:
df['post_mentions'] = pd.DataFrame(df['post_mentions'].str.replace('"', ''))

In [71]:
df['post_mentions'] = pd.DataFrame(df['post_mentions'].str.replace('"",', ''))

In [72]:
df['post_mentions'] = pd.DataFrame(df['post_mentions'].str.replace("#<Hashie::Array \[]>", ' '))

In [73]:
df.post_mentions = df.post_mentions.astype(str)

## Clean Text Data and save to a CSV

In [74]:
df_text = df[['id', 'description', 'post_caption', 'post_hashtags', 'post_mentions']]

In [75]:
df_text['all_text'] = df_text[['post_caption', 'post_hashtags', 'post_mentions' ]].apply(lambda x: ' '.join(x), axis=1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Then groupby user id and description.  This allows for the description to be entered just once. 

In [76]:
df_text = df_text.groupby(['id', 'description'])['all_text'].apply(' '.join).reset_index()

In [77]:
df_text['all_text'] = df_text[['description', 'all_text']].apply(lambda x: ' '.join(x), axis=1)

In [84]:
df_text.drop('description', axis=1, inplace=True)

In [80]:
df_text.all_text = pd.DataFrame(df_text['all_text'].str.replace('\n-\n', ' '))

In [81]:
bad_character_list = ['\\n•', '\\n', '\n', ',', '.', '?', '!', ')', '(', '#', '&', '\r', '"', '\r\n\r\n']

,id,all_text
0,17,Wow so fashion Every pup is born with the ma...
1,21,Personal Style Blog Taking In The Calm Before...
2,28,Digital Influencer sharing her lifestyle based...
3,30,So were you on the naughty list or the nice ...
4,65,\u1F6D2 ONLINE 📍wwwkm33officialcom 1 de...


In [82]:
for symbol in bad_character_list:
    df_text.all_text = pd.DataFrame(df_text['all_text'].str.replace(symbol, '', regex=False))

### Save Text data to CSV

In [85]:
df_text.to_csv('text_per_user.csv')

## Save Num Values to CSV

We will save to two CSV files, one which includes the text columns and the second which only includes the integer based columns. We are doing this to allow for an intermediate step of Natural Language Processing before combining to a single Cosine Similarity recommendation. 

In [90]:
df.fillna(value=0, inplace=True)

In [91]:
df_values = df[['id', 'birthday', 'gender', 'post_followers',
       'post_likes', 'post_interactions', 'post_comments',
       'no_cat', 'cat_beauty', 'cat_photography', 'cat_illustration', 'cat_vlog', 'cat_food',
       'cat_artist', 'cat_musician/DJ', 'cat_tech', 'cat_parenting',
       'cat_entertainment', 'cat_fitness', 'cat_diy', 'cat_health_wellness',
       'cat_automotive', 'cat_home_decor', 'cat_art_design', 'cat_travel',
       'cat_fashion', 'cat_lifestyle', 'cat_menswear', 'cat_models', 'unknown',
       'cat_news', 'cat_personal_style', 'post_day', 'post_hour']]

Group all by ID and set all values to **Mean**

In [92]:
df_values = df.groupby(df_values.id).mean().round()

### Save Text data to CSV

In [90]:
df_values.to_csv('values_per_user.csv')